In [1]:
# %%capture
# ! pip install pytorch-lightning
# ! pip install pytorch-lightning-bolts
# ! pip install torchvision

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pl_bolts.datasets import DummyDataset
from torch.optim import Adam
from torch.utils.data import TensorDataset

/Users/shao/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  f' install it with `pip install {pypi_name}`.' + extra_text
/Users/shao/anaconda3/lib/python3.7/site-packages/pl_bolts/utils/warnings.py:32: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  f' install it with `pip install {pypi_name}`.' + extra_text


# Multi-Task Learning

### Simple Linear Regression

In [3]:
# Input (temp, rainfall, humidity)
input_1 = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37]], 
                  dtype='float32')
# Targets (apples, oranges)
target_1 = np.array([[52], [84], [115], 
                    [23], [102], [59], 
                    [85], [115], [27], 
                    [101], [51], [82], 
                    [112], [23]], 
                   dtype='float32')

input_1 = torch.from_numpy(input_1)
target_1 = torch.from_numpy(target_1)

train_1 = TensorDataset(input_1, target_1)
train_1[0:3]

model1 = nn.Linear(3, 1)

# Parameters
params = list(model1.parameters())

# Define Loss
import torch.nn.functional as F
loss_fn = F.mse_loss

# Define optimizer
opt = torch.optim.SGD(params, lr=1e-5)

def fit(num_epochs, model1, loss_fn, opt, train_dl1):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl1:
            
            # 1. Generate predictions
            pred1 = model1(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred1, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

#fit model for 100 epochs
fit(100, model1, loss_fn ,opt ,train_1)

Epoch [10/100], Loss: 10.8705
Epoch [20/100], Loss: 1.1721
Epoch [30/100], Loss: 0.4507
Epoch [40/100], Loss: 0.2477
Epoch [50/100], Loss: 0.1459
Epoch [60/100], Loss: 0.0881
Epoch [70/100], Loss: 0.0543
Epoch [80/100], Loss: 0.0343
Epoch [90/100], Loss: 0.0223
Epoch [100/100], Loss: 0.0150


In [4]:
list(model1.parameters())

[Parameter containing:
 tensor([[-0.3626,  0.7432,  0.7699]], requires_grad=True),
 Parameter containing:
 tensor([-0.3707], requires_grad=True)]

### Double-Task Linear Regression

In [107]:
# Input (temp, rainfall, humidity)
input_1 = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37]], 
                  dtype='float32')
# Targets (apples, oranges)
target_1 = np.array([[52], [84], [115], 
                    [23], [102], [59], 
                    [85], [115], [27], 
                    [101], [51], [82], 
                    [112], [23]], 
                   dtype='float32')

input_2 = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37]], 
                  dtype='float32')
# Targets (apples, oranges)
target_2 = np.array([[52], [84], [115], 
                    [23], [102], [59], 
                    [85], [115], [27], 
                    [101], [51], [82], 
                    [112], [23]], 
                   dtype='float32')

input_1 = torch.from_numpy(input_1)
target_1 = torch.from_numpy(target_1)

train_1 = TensorDataset(input_1, target_1)
train_1[0:3]

input_2 = torch.from_numpy(input_2)
target_2 = torch.from_numpy(target_2)

train_2 = TensorDataset(input_2, target_2)
train_2[0:3]


model1 = nn.Linear(3, 1)
model2 = nn.Linear(3, 1)

##### add another parameters
# Parameters
params = list(model2.parameters()) + list(model1.parameters())

# Define Loss
import torch.nn.functional as F
loss_fn = F.mse_loss

# Define optimizer
opt = torch.optim.SGD(params, lr=1e-5)

def fit(num_epochs, model1, model2, loss_fn, opt, train_dl1, train_dl2):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        ##### add another data
        # Train with batches of data
        for (xb, yb), (xa, ya) in zip(train_dl2 ,train_dl1):
            
            # 1. Generate predictions
            pred1 = model1(xa)
            pred2 = model2(xb)
            
            ##### l1 norm
            regularize_term_1 = sum(p.abs().sum() for p in model1.parameters())
            regularize_term_2 = sum(p.abs().sum() for p in model2.parameters())
            
            ##### edit loss
            # 2. Calculate loss
            loss = loss_fn(pred1, ya)+ loss_fn(pred2, yb) + regularize_term_1+regularize_term_2
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

#fit model for 100 epochs
fit(1000, model1, model2, loss_fn ,opt ,train_1, train_2)

Epoch [100/1000], Loss: 4.3452
Epoch [200/1000], Loss: 4.2779
Epoch [300/1000], Loss: 4.2494
Epoch [400/1000], Loss: 4.2212
Epoch [500/1000], Loss: 4.1931
Epoch [600/1000], Loss: 4.1650
Epoch [700/1000], Loss: 4.1369
Epoch [800/1000], Loss: 4.1088
Epoch [900/1000], Loss: 4.0872
Epoch [1000/1000], Loss: 4.0777


In [108]:
print(loss_fn(pred1, target_1))
print(loss_fn(pred2, target_2))

tensor(7.4928, grad_fn=<MseLossBackward>)
tensor(7.4592, grad_fn=<MseLossBackward>)


In [109]:
list(model2.parameters())

[Parameter containing:
 tensor([[-0.3657,  0.7394,  0.7804]], requires_grad=True),
 Parameter containing:
 tensor([-0.2826], requires_grad=True)]

In [110]:
list(model1.parameters())

[Parameter containing:
 tensor([[-0.3680,  0.7394,  0.7789]], requires_grad=True),
 Parameter containing:
 tensor([-0.0001], requires_grad=True)]